# MLflow with recipe

In this second notebook we will take the same example than the first example but we will use MLflow recipe to accomplish the same result. 
We will go throught the same steps than on the first notebook but this time we will use the MLflow recipe module. 

Like on the previous notebook, you will have some tasks that need to be completed. You will be able to find where they are in the code by searching for `# ToDo#: ...`

In this notebook, you will be asked to:
* ToDo1: Add a column to indicate if the wine is red or white
* ToDo2: specify split ratios for train, validation, and test sets
* ToDo3: Create a Pipeline object that transforms the features
* ToDo4: Create a LinearRegression estimator with the estimator_params
* ToDo5: add custom metrics to our recipe
* ToDo6: look in the UI what did the recipe logged by default. What was added compared with last notebook?
* ToDo7: change the model uri to the one from current run
* ToDo8: query the model with some test data
* ToDo9: [To Go Further] use the AutoML estimator instead and use the UI to compare the results
* ToDo10: [To Go Further] use databricks mlflow instead of local mlflow server

If you need help you can browse through the following documentation:
* [MLflow](https://mlflow.org/docs/latest/index.html), in particular the [recipe module](https://mlflow.org/docs/latest/recipes.html)
* [MLflow recipe template](https://github.com/mlflow/recipes-regression-template)
* [MLflow recipe example](https://github.com/mlflow/recipes-examples)

In [1]:
from mlflow.recipes import Recipe
import os


In [3]:
# Note: please change the directory if you are not using a dev container.
# We want to have the working directory to be the src folder in the mlflow-trainng repo
os.chdir("/workspaces/mlflow-training/src")


FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/mlflow-training/src'

In [ ]:
r = Recipe(profile="local")


2023/05/31 19:40:05 INFO mlflow.recipes.recipe: Creating MLflow Recipe 'src' with profile: 'local'


In [4]:
r.clean()


In [5]:
# for some reason you might have to run the cell twice before working
r.inspect()


## Ingest data

In [6]:
!cat steps/ingest.py

import pandas as pd
from pandas import DataFrame


def load_file_as_dataframe(file_path: str, file_format: str) -> DataFrame:
    """Load a csv file as a dataframe and add a column to indicate if the wine is red or white"""
    df = pd.read_csv(file_path, sep=";")
    # ToDo1: Add a column to indicate if the wine is red or white
    df["is_red"] = 1 if "red" in str(file_path) else 0
    return df


In [7]:
r.run("ingest")


2023/05/31 19:40:13 INFO mlflow.recipes.step: Running step ingest...


## Split data

We want to split the data to have the following proportion:
- 80% training
- 10% evaluation
- 10% test

In [8]:
!cat recipe.yaml

recipe: "regression/v1"
target_col: "quality"
primary_metric: "root_mean_squared_error"
steps:
  ingest: {{INGEST_CONFIG}}
  split:
    # ToDo2: specify split ratios for train, validation, and test sets
    split_ratios: [0.8, 0.1, 0.1]
  transform:
    using: custom
    transformer_method: transformer_fn
  train:
    using: custom
    estimator_method: estimator_fn
    # # ToDo9: use the AutoML estimator
    # commment line 13 and 15 and uncomment line 17 and 18
    # using: "automl/flaml"
    # time_budget_secs: 60
  evaluate:
    validation_criteria:
      - metric: root_mean_squared_error
        threshold: 1
  register:
    allow_non_validated_model: false
  ingest_scoring: {{INGEST_SCORING_CONFIG}}
  predict:
    output: {{PREDICT_OUTPUT_CONFIG}}

# ToDo5: add custom metrics to our recipe
custom_metrics:
  - name: rounded_root_mean_squared_error
    function: rounded_root_mean_squared_error
    greater_is_better: False

In [9]:
r.run("split")


2023/05/31 19:40:16 INFO mlflow.recipes.utils.execution: ingest: No changes. Skipping.


Run MLFlow Recipe step: split
2023/05/31 19:40:18 INFO mlflow.recipes.step: Running step split...


## Transform data

In [10]:
!cat steps/transform.py

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def transformer_fn():
    """
    Returns a Pipeline object that transforms the features
    """
    # ToDo3: Create a Pipeline object that transforms the features
    columns = [
        "fixed acidity",
        "volatile acidity",
        "citric acid",
        "residual sugar",
        "chlorides",
        "free sulfur dioxide",
        "total sulfur dioxide",
        "density",
        "pH",
        "sulphates",
        "alcohol",
        "is_red",
    ]
    pipeline = Pipeline(
        [
            (
                "ct",
                ColumnTransformer(
                    [
                        (
                            "minmax",
                            StandardScaler(),
                            columns,
                        ),
                    ]
                ),
            )
        ]
    )
    return pipeline


In [11]:
r.run("transform")


2023/05/31 19:40:20 INFO mlflow.recipes.utils.execution: ingest, split: No changes. Skipping.


Run MLFlow Recipe step: transform
2023/05/31 19:40:23 INFO mlflow.recipes.step: Running step transform...


## Train model

In [12]:
!cat steps/train.py

from typing import Any, Dict

from sklearn.linear_model import LinearRegression


def estimator_fn(estimator_params: Dict[str, Any] = None):
    # ToDo4: Create a LinearRegression estimator with the estimator_params
    if estimator_params is None:
        estimator_params = {}
    return LinearRegression(**estimator_params)


In [13]:
!cat recipe.yaml

recipe: "regression/v1"
target_col: "quality"
primary_metric: "root_mean_squared_error"
steps:
  ingest: {{INGEST_CONFIG}}
  split:
    # ToDo2: specify split ratios for train, validation, and test sets
    split_ratios: [0.8, 0.1, 0.1]
  transform:
    using: custom
    transformer_method: transformer_fn
  train:
    using: custom
    estimator_method: estimator_fn
    # # ToDo9: use the AutoML estimator
    # commment line 13 and 15 and uncomment line 17 and 18
    # using: "automl/flaml"
    # time_budget_secs: 60
  evaluate:
    validation_criteria:
      - metric: root_mean_squared_error
        threshold: 1
  register:
    allow_non_validated_model: false
  ingest_scoring: {{INGEST_SCORING_CONFIG}}
  predict:
    output: {{PREDICT_OUTPUT_CONFIG}}

# ToDo5: add custom metrics to our recipe
custom_metrics:
  - name: rounded_root_mean_squared_error
    function: rounded_root_mean_squared_error
    greater_is_better: False

In [14]:
r.run("train")


2023/05/31 19:40:26 INFO mlflow.recipes.utils.execution: ingest, split, transform: No changes. Skipping.


Run MLFlow Recipe step: train
2023/05/31 19:40:28 INFO mlflow.recipes.step: Running step train...
2023/05/31 19:40:34 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/05/31 19:40:56 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/05/31 19:40:57 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.


In [15]:
r.run("evaluate")

2023/05/31 19:41:00 INFO mlflow.recipes.utils.execution: ingest, split, transform, train: No changes. Skipping.


Run MLFlow Recipe step: evaluate
2023/05/31 19:41:04 INFO mlflow.recipes.step: Running step evaluate...
2023/05/31 19:41:07 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/05/31 19:41:09 INFO mlflow.models.evaluation.default_evaluator: Shap explainer _PatchedKernelExplainer is used.

100%|██████████| 10/10 [00:01<00:00,  5.15it/s]
elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
2023/05/31 19:41:18 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.


In [16]:
r.run("register")


2023/05/31 19:41:19 INFO mlflow.recipes.utils.execution: ingest, split, transform, train, evaluate: No changes. Skipping.


Run MLFlow Recipe step: register
2023/05/31 19:41:22 INFO mlflow.recipes.step: Running step register...
Registered model 'red_wine_scorer' already exists. Creating a new version of this model...
2023/05/31 19:41:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: red_wine_scorer, version 10
Created version '10' of model 'red_wine_scorer'.


In [17]:
print("If you shut down mlflow server from notebook 01")
print(
    "Please copy the command below in a new terminal on your IDE and let it run until the end of the notebook \n"
)

print("mlflow server \\")
print("    --backend-store-uri sqlite:///src/metadata/mlflow/mlruns.db \\")
print("    --default-artifact-root ./src/metadata/mlflow/mlartifacts \\")
print("    --host 0.0.0.0 \\")
print("    --port 5000")

# ToDo6: look in the UI what did the recipe logged by default. What was added compared with last notebook?


If you shut down mlflow server from notebook 01
Please copy the command below in a new terminal on your IDE and let it run until the end of the notebook 

mlflow server \
    --backend-store-uri sqlite:///src/metadata/mlflow/mlruns.db \
    --default-artifact-root ./src/metadata/mlflow/mlartifacts \
    --host 0.0.0.0 \
    --port 5000


## Predict with trained model

### Predict on batch inference

In [18]:
# Notes: it takes around 5 minutes to run...
# we can only run it locally, if you are using codespace it will break your environemt
# if "GITHUB_CODESPACE_TOKEN" not in os.environ:
#     r.run("predict")


Run MLFlow Recipe step: ingest_scoring
2023/05/31 19:41:26 INFO mlflow.recipes.step: Running step ingest_scoring...
Run MLFlow Recipe step: predict
2023/05/31 19:41:30 INFO mlflow.recipes.step: Running step predict...
2023/05/31 19:41:31 INFO mlflow.recipes.steps.predict: Creating new spark session
:: loading settings :: url = jar:file:/home/nonroot/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/nonroot/.ivy2/cache
The jars for the packages stored in: /home/nonroot/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-df7fc117-89f9-4441-bfce-1c43a33318c3;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.2.1 in central
	found io.delta#delta-storage;1.2.1 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
downloading https://repo1.maven.org/maven2/io/d

### Predict in real time

We can also use the mlflow model to do rediction in real-time. To do so we will need to:
1. run an mlflow server to be able to distribute the model (like in notebook 01)
2. create a serving enpoint which will pull the model from mlflow server
3. finally we can query our model in real time using `curl`

In [19]:
print("Please copy the command below in a new terminal on your IDE \n")

print("MLFLOW_TRACKING_URI=http://0.0.0.0:5000 mlflow models serve \\")
print("      --host=0.0.0.0 \\")
print("      --port=5011 \\")
print("      --env-manager=local \\")
# ToDo7: change the model uri to the one from current run
print(f"      --model-uri runs:/{r.get_artifact('run').info.run_id}/train/model/")


Please copy the command below in a new terminal on your IDE 

MLFLOW_TRACKING_URI=http://0.0.0.0:5000 mlflow models serve \
      --host=0.0.0.0 \
      --port=5011 \
      --env-manager=local \
      --model-uri runs:/6ac93f0a4dc146809d3faef550df387b/train/model/


In [20]:
# ToDo8: query the model with some test data
test_data = r.get_artifact("test_data")
request_data = test_data.iloc[0:4].to_json(orient="records")
print("You can copy the command below on one of your terminal \n")
print(
    """curl http://0.0.0.0:5011/invocations -H 'Content-Type: application/json' -d '{"dataframe_records": """
    + request_data
    + """}'"""
)


You can copy the command below on one of your terminal 

curl http://0.0.0.0:5011/invocations -H 'Content-Type: application/json' -d '{"dataframe_records": [{"fixed acidity":7.8,"volatile acidity":0.76,"citric acid":0.04,"residual sugar":2.3,"chlorides":0.092,"free sulfur dioxide":15.0,"total sulfur dioxide":54.0,"density":0.997,"pH":3.26,"sulphates":0.65,"alcohol":9.8,"quality":5,"is_red":1},{"fixed acidity":7.6,"volatile acidity":0.39,"citric acid":0.31,"residual sugar":2.3,"chlorides":0.082,"free sulfur dioxide":23.0,"total sulfur dioxide":71.0,"density":0.9982,"pH":3.52,"sulphates":0.65,"alcohol":9.7,"quality":5,"is_red":1},{"fixed acidity":6.3,"volatile acidity":0.39,"citric acid":0.16,"residual sugar":1.4,"chlorides":0.08,"free sulfur dioxide":11.0,"total sulfur dioxide":23.0,"density":0.9955,"pH":3.34,"sulphates":0.56,"alcohol":9.3,"quality":5,"is_red":1},{"fixed acidity":7.5,"volatile acidity":0.49,"citric acid":0.2,"residual sugar":2.6,"chlorides":0.332,"free sulfur dioxide":8

## To Go Further

You can try to use `flaml` to get one of the best model. 

In [21]:
# ToDo9: [To Go Further] use the AutoML estimator instead and use the UI to compare the results
!cat recipe.yaml

recipe: "regression/v1"
target_col: "quality"
primary_metric: "root_mean_squared_error"
steps:
  ingest: {{INGEST_CONFIG}}
  split:
    # ToDo2: specify split ratios for train, validation, and test sets
    split_ratios: [0.8, 0.1, 0.1]
  transform:
    using: custom
    transformer_method: transformer_fn
  train:
    # using: custom
    # estimator_method: estimator_fn
    # # ToDo9: use the AutoML estimator
    # commment line 13 and 15 and uncomment line 17 and 18
    using: "automl/flaml"
    time_budget_secs: 60
  evaluate:
    validation_criteria:
      - metric: root_mean_squared_error
        threshold: 1
  register:
    allow_non_validated_model: false
  ingest_scoring: {{INGEST_SCORING_CONFIG}}
  predict:
    output: {{PREDICT_OUTPUT_CONFIG}}

# ToDo5: add custom metrics to our recipe
custom_metrics:
  - name: rounded_root_mean_squared_error
    function: rounded_root_mean_squared_error
    greater_is_better: False

In [22]:

r.run("register")


2023/05/31 19:47:45 INFO mlflow.recipes.utils.execution: ingest, split, transform: No changes. Skipping.


Run MLFlow Recipe step: train
2023/05/31 19:47:48 INFO mlflow.recipes.step: Running step train...
[flaml.automl.logger: 05-31 19:47:50] {1693} INFO - task = regression
[flaml.automl.logger: 05-31 19:47:50] {1700} INFO - Data split method: uniform
[flaml.automl.logger: 05-31 19:47:50] {1703} INFO - Evaluation method: cv
[flaml.automl.logger: 05-31 19:47:50] {1801} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 05-31 19:47:50] {1911} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 05-31 19:47:50] {2221} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 05-31 19:47:50] {2347} INFO - Estimated sufficient time budget=985s. Estimated necessary time budget=7s.
[flaml.automl.logger: 05-31 19:47:51] {2394} INFO -  at 0.1s,	estimator lgbm's best error=0.8100,	best estimator lgbm's best error=0.8100
[flaml.automl.logger: 05-31 19:47:51] {2221} INFO - iteration 1, current learner lgbm
[flaml.automl.l

In [23]:
print("Please copy the command below in a new terminal on your IDE \n")

print("MLFLOW_TRACKING_URI=http://0.0.0.0:5000 mlflow models serve \\")
print("      --host=0.0.0.0 \\")
print("      --port=5012 \\")
print("      --env-manager=local \\")
print(f"      --model-uri runs:/{r.get_artifact('run').info.run_id}/train/model/")


Please copy the command below in a new terminal on your IDE 

MLFLOW_TRACKING_URI=http://0.0.0.0:5000 mlflow models serve \
      --host=0.0.0.0 \
      --port=5012 \
      --env-manager=local \
      --model-uri runs:/e7d60ee80e5a49539f6e5e02403902b8/train/model/


In [24]:
test_data = r.get_artifact("test_data")
request_data = test_data.iloc[0:4].to_json(orient="records")
print("You can copy the command below on one of your terminal \n")
print(
    """curl http://0.0.0.0:5012/invocations -H 'Content-Type: application/json' -d '{"dataframe_records": """
    + request_data
    + """}'"""
)


You can copy the command below on one of your terminal 

curl http://0.0.0.0:5012/invocations -H 'Content-Type: application/json' -d '{"dataframe_records": [{"fixed acidity":7.8,"volatile acidity":0.76,"citric acid":0.04,"residual sugar":2.3,"chlorides":0.092,"free sulfur dioxide":15.0,"total sulfur dioxide":54.0,"density":0.997,"pH":3.26,"sulphates":0.65,"alcohol":9.8,"quality":5,"is_red":1},{"fixed acidity":7.6,"volatile acidity":0.39,"citric acid":0.31,"residual sugar":2.3,"chlorides":0.082,"free sulfur dioxide":23.0,"total sulfur dioxide":71.0,"density":0.9982,"pH":3.52,"sulphates":0.65,"alcohol":9.7,"quality":5,"is_red":1},{"fixed acidity":6.3,"volatile acidity":0.39,"citric acid":0.16,"residual sugar":1.4,"chlorides":0.08,"free sulfur dioxide":11.0,"total sulfur dioxide":23.0,"density":0.9955,"pH":3.34,"sulphates":0.56,"alcohol":9.3,"quality":5,"is_red":1},{"fixed acidity":7.5,"volatile acidity":0.49,"citric acid":0.2,"residual sugar":2.6,"chlorides":0.332,"free sulfur dioxide":8

In [25]:
# ToDo10: [To Go Further] use databricks mlflow instead of local mlflow server
# Note: you will need to have a databricks community account (free)
# See ANNEXE.md for more details
